# 04 — Tail Index & Persistence Analysis

This notebook provides a comprehensive tour of **13 estimators** for measuring
tail heaviness and serial persistence in financial return series.

**Tail-index estimators** (higher alpha / lower gamma = thinner tail):

| # | Estimator | What it measures |
|---|-----------|------------------|
| 1 | Hill | Tail index alpha via top-order statistics |
| 2 | Kappa (max-stability) | Deviation of maxima scaling from Gaussian benchmark |
| 3 | Taleb kappa | Ratio-based kappa using two sample sizes |
| 4 | Pickands | Non-parametric EVI using three quantiles |
| 5 | DEH (Dekkers-Einmahl-de Haan) | Moment estimator of the extreme-value index |
| 6 | QQ | Tail index from the slope of a Pareto QQ plot |
| 7 | Max-to-Sum ratio | Concentration: max observation / total sum |

**Persistence / long-memory estimators** (H > 0.5 or d > 0 = persistence):

| # | Estimator | What it measures |
|---|-----------|------------------|
| 8 | Hurst exponent | Rescaled-range persistence (H) |
| 9 | DFA | Detrended fluctuation analysis scaling exponent |
| 10 | Spectral | Long-memory parameter d from low-frequency periodogram |

Plus **rolling** variants for Hill, Kappa, and Taleb kappa (sections 4-6),
giving us 13 distinct estimator functions in total.

All estimators are implemented in Rust and exposed via `fatcrash._core`.

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fatcrash.data.ingest import from_sample, from_csv, load_fred_forex
from fatcrash.data.transforms import log_returns, negative_returns, block_maxima

from fatcrash._core import (
    hill_estimator, hill_rolling,
    kappa_metric, kappa_rolling,
    taleb_kappa, taleb_kappa_rolling,
    pickands_estimator, pickands_rolling,
    hurst_exponent, hurst_rolling,
    dfa_exponent, dfa_rolling,
    deh_estimator, deh_rolling,
    qq_estimator, qq_rolling,
    maxsum_ratio, maxsum_rolling,
    spectral_exponent, spectral_rolling,
)

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (14, 5)

## 1. Load data

In [ ]:
df = from_sample("btc")
returns = log_returns(df)          # numpy array, length = len(df) - 1
losses = negative_returns(returns) # positive magnitudes of down days
dates = df.index[1:]              # align with returns

print(f"BTC observations : {len(df)}")
print(f"Return series    : {len(returns)}")
print(f"Loss days        : {len(losses)}")
print(f"Date range       : {df.index[0].date()} to {df.index[-1].date()}")

## 2. Hill estimator

The **Hill estimator** for the tail index $\alpha$ uses the $k$ largest
order statistics $X_{(1)} \geq X_{(2)} \geq \cdots \geq X_{(k)}$:

$$\hat{\alpha}_k^{\,-1} = \frac{1}{k} \sum_{i=1}^{k} \ln X_{(i)} - \ln X_{(k+1)}$$

Interpretation:
- $\alpha \leq 1$: infinite mean
- $\alpha \leq 2$: infinite variance (extreme fat tails)
- $2 < \alpha < 4$: finite variance but heavy tail
- $\alpha \geq 4$: moderately heavy tail

In [ ]:
alpha_hill = hill_estimator(losses)

print(f"Hill tail index (alpha): {alpha_hill:.3f}")
if alpha_hill < 2:
    print("  => Infinite variance regime")
elif alpha_hill < 4:
    print("  => Finite variance, heavy tail")
else:
    print("  => Moderately heavy tail")

## 3. Hill plot

The Hill plot shows $\hat{\alpha}$ as a function of $k$ (number of order
statistics used). A stable plateau indicates a reliable estimate:
- Small $k$: high variance (too few observations)
- Large $k$: high bias (non-tail data enters the estimate)

In [ ]:
k_max = int(0.15 * len(losses))
k_values = np.arange(10, k_max, 5)
alpha_values = np.array([hill_estimator(losses, k=int(kv)) for kv in k_values])

fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(k_values, alpha_values, color="steelblue", linewidth=0.8)
ax.axhline(alpha_hill, color="red", linestyle="--", alpha=0.6,
           label=f"Default alpha = {alpha_hill:.2f}")
ax.axhline(2, color="gray", linestyle=":", alpha=0.5, label="alpha = 2 (infinite var)")
ax.set_xlabel("k (number of order statistics)")
ax.set_ylabel("Estimated tail index alpha")
ax.set_title("Hill Plot: BTC Losses")
ax.set_ylim(0, 8)
ax.legend()
plt.tight_layout()
plt.show()

## 4. Rolling tail index

The tail index is not constant -- it evolves as market conditions change.
A declining $\alpha$ signals increasing crash risk. We use a 500-day
rolling window.

In [ ]:
window = 500
rolling_alpha = hill_rolling(returns, window=window)

# rolling array has length = len(returns) - window + 1
roll_dates = dates[window - 1:]

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(df.index, df["close"], color="steelblue", linewidth=0.8)
axes[0].set_yscale("log")
axes[0].set_ylabel("Price (USD, log scale)")
axes[0].set_title("BTC/USD Price")

axes[1].plot(roll_dates, rolling_alpha, color="darkorange", linewidth=1)
axes[1].axhline(2, color="red", linestyle="--", alpha=0.5,
               label="alpha = 2 (infinite variance)")
axes[1].axhline(3, color="gray", linestyle="--", alpha=0.5, label="alpha = 3")
axes[1].set_ylabel("Tail index (alpha)")
axes[1].set_title(f"Rolling Hill Tail Index ({window}-day window)")
axes[1].set_ylim(0, 6)
axes[1].legend()

plt.tight_layout()
plt.show()

## 5. Kappa metric (max-stability)

The **kappa metric** measures how the sample maximum scales with subsample
size compared to a Gaussian benchmark. A higher kappa indicates fatter tails.

Returns `(kappa, benchmark)` where `benchmark` is the Gaussian reference value.

In [ ]:
kappa_val, kappa_bench = kappa_metric(returns, n_subsamples=10)

print(f"Kappa          : {kappa_val:.4f}")
print(f"Benchmark (Gaussian): {kappa_bench:.4f}")
print()
print("Interpretation:")
print("  kappa ~ benchmark => Gaussian-like tail")
print("  kappa > benchmark => Heavier than Gaussian")
print("  kappa >> benchmark => Extremely heavy tail")

## 6. Rolling kappa

Track how kappa evolves over time. Spikes in kappa signal tail-thickening
regimes.

In [ ]:
rolling_kap, kap_bench = kappa_rolling(returns, window=window, n_subsamples=10)
kap_dates = dates[window - 1:]

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(df.index, df["close"], color="steelblue", linewidth=0.8)
axes[0].set_yscale("log")
axes[0].set_ylabel("Price (USD, log scale)")
axes[0].set_title("BTC/USD Price")

axes[1].plot(kap_dates, rolling_kap, color="darkorange", linewidth=1,
            label="Rolling kappa")
axes[1].axhline(kap_bench, color="red", linestyle="--", alpha=0.5,
               label=f"Gaussian benchmark = {kap_bench:.3f}")
axes[1].set_ylabel("Kappa")
axes[1].set_title(f"Rolling Kappa Metric ({window}-day window)")
axes[1].legend()

plt.tight_layout()
plt.show()

## 7. Alternative Tail Estimators

Beyond Hill, several other estimators target the extreme-value index (EVI)
or tail thickness from different angles:

- **DEH (Dekkers-Einmahl-de Haan):** A moment-based estimator of the EVI gamma.
  Works for all tail types (heavy, light, short). gamma > 0 means heavy tail.
- **QQ estimator:** Fits the slope of a Pareto QQ plot to the upper-order
  statistics. Returns an alpha estimate (like Hill).
- **Pickands estimator:** Uses three quantiles at positions k, 2k, 4k to
  estimate the EVI gamma non-parametrically. gamma > 0 means heavy tail.
- **Max-to-Sum ratio:** The ratio max(|X|) / sum(|X|). For Gaussian data
  this converges to 0; for heavy-tailed data it stays positive.
- **Taleb kappa:** Compares moment ratios at two sample sizes n0 and n1.
  Returns (kappa, benchmark).

In [ ]:
# Compute all alternative tail estimators on BTC losses
deh_gamma   = deh_estimator(losses)
qq_alpha    = qq_estimator(losses)
pickands_g  = pickands_estimator(losses)
maxsum_val  = maxsum_ratio(losses)
taleb_k, taleb_bench = taleb_kappa(returns)

tail_results = pd.DataFrame([
    {"Estimator": "Hill",        "Statistic": "alpha", "Value": alpha_hill,
     "Interpretation": f"Tail index; < 2 = infinite variance"},
    {"Estimator": "Kappa",       "Statistic": "kappa", "Value": kappa_val,
     "Interpretation": f"Max-stability; benchmark = {kappa_bench:.3f}"},
    {"Estimator": "Taleb kappa", "Statistic": "kappa", "Value": taleb_k,
     "Interpretation": f"Ratio-based; benchmark = {taleb_bench:.3f}"},
    {"Estimator": "DEH",         "Statistic": "gamma", "Value": deh_gamma,
     "Interpretation": "> 0 = heavy tail; = 1/alpha"},
    {"Estimator": "QQ",          "Statistic": "alpha", "Value": qq_alpha,
     "Interpretation": "Pareto QQ slope; comparable to Hill"},
    {"Estimator": "Pickands",    "Statistic": "gamma", "Value": pickands_g,
     "Interpretation": "> 0 = heavy tail; non-parametric EVI"},
    {"Estimator": "Max-to-Sum",  "Statistic": "ratio", "Value": maxsum_val,
     "Interpretation": "Concentration; > 0 in heavy tails, -> 0 for Gaussian"},
])

print("BTC Tail Estimator Comparison")
print("=" * 80)
display(tail_results.style.format({"Value": "{:.4f}"}).hide(axis="index"))

In [ ]:
# Grouped bar chart: alpha-type estimators vs gamma-type estimators
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Panel 1: Alpha estimators (Hill, QQ)
alpha_est = {"Hill": alpha_hill, "QQ": qq_alpha}
bars = axes[0].bar(alpha_est.keys(), alpha_est.values(), color=["steelblue", "seagreen"],
                   alpha=0.8, edgecolor="black", linewidth=0.5)
axes[0].axhline(2, color="red", linestyle="--", alpha=0.5, label="alpha=2")
axes[0].set_ylabel("Alpha (tail index)")
axes[0].set_title("Tail Index Estimators")
axes[0].legend()
for bar, val in zip(bars, alpha_est.values()):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                f"{val:.2f}", ha="center", va="bottom", fontsize=10)

# Panel 2: Gamma estimators (DEH, Pickands)
gamma_est = {"DEH": deh_gamma, "Pickands": pickands_g}
bars = axes[1].bar(gamma_est.keys(), gamma_est.values(), color=["coral", "orchid"],
                   alpha=0.8, edgecolor="black", linewidth=0.5)
axes[1].axhline(0, color="gray", linestyle="-", alpha=0.3)
axes[1].set_ylabel("Gamma (EVI)")
axes[1].set_title("Extreme Value Index Estimators")
for bar, val in zip(bars, gamma_est.values()):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                f"{val:.3f}", ha="center", va="bottom", fontsize=10)

# Panel 3: Kappa and Max-to-Sum
other_est = {"Kappa": kappa_val, "Taleb K": taleb_k, "MaxSum": maxsum_val}
bars = axes[2].bar(other_est.keys(), other_est.values(),
                   color=["darkorange", "gold", "mediumpurple"],
                   alpha=0.8, edgecolor="black", linewidth=0.5)
axes[2].set_ylabel("Value")
axes[2].set_title("Kappa & Concentration")
for bar, val in zip(bars, other_est.values()):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002,
                f"{val:.3f}", ha="center", va="bottom", fontsize=10)

plt.suptitle("BTC: All Tail Estimators", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 8. Persistence Analysis

Persistence estimators detect **long-range dependence** in the return series.
If returns (or absolute returns) exhibit persistence, past shocks continue
to influence future behavior.

- **Hurst exponent (H):** From rescaled-range (R/S) analysis.
  H = 0.5 means random walk; H > 0.5 means persistence; H < 0.5 means mean-reversion.
- **DFA exponent (alpha_DFA):** Detrended fluctuation analysis. Same interpretation
  as Hurst but more robust to trends and non-stationarity. alpha_DFA ~ 0.5 = no memory.
- **Spectral exponent (d):** Estimated from the periodogram at low frequencies.
  d > 0 means long memory; d = 0 means short memory; d < 0 means antipersistence.

In [ ]:
hurst_h   = hurst_exponent(returns)
dfa_alpha = dfa_exponent(returns)
spec_d    = spectral_exponent(returns)

# Also compute on absolute returns (volatility clustering)
abs_ret = np.abs(returns)
hurst_abs = hurst_exponent(abs_ret)
dfa_abs   = dfa_exponent(abs_ret)
spec_abs  = spectral_exponent(abs_ret)

persist_df = pd.DataFrame([
    {"Estimator": "Hurst H",     "Returns": hurst_h,   "|Returns|": hurst_abs,
     "Neutral": 0.50, "Interpretation": "> 0.5 = persistent"},
    {"Estimator": "DFA alpha",   "Returns": dfa_alpha,  "|Returns|": dfa_abs,
     "Neutral": 0.50, "Interpretation": "> 0.5 = persistent"},
    {"Estimator": "Spectral d",  "Returns": spec_d,     "|Returns|": spec_abs,
     "Neutral": 0.00, "Interpretation": "> 0 = long memory"},
])

print("BTC Persistence Analysis")
print("=" * 70)
display(persist_df.style.format({
    "Returns": "{:.4f}", "|Returns|": "{:.4f}", "Neutral": "{:.2f}"
}).hide(axis="index"))

print()
print("Note: Returns typically show weak persistence (H ~ 0.5).")
print("Absolute returns usually show strong persistence (H >> 0.5),")
print("reflecting volatility clustering.")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = np.arange(3)
width = 0.35
labels = ["Hurst H", "DFA alpha", "Spectral d"]
vals_ret = [hurst_h, dfa_alpha, spec_d]
vals_abs = [hurst_abs, dfa_abs, spec_abs]

# Panel 1: Raw returns
bars1 = axes[0].bar(x, vals_ret, width, color="steelblue", alpha=0.8,
                    edgecolor="black", linewidth=0.5, label="Returns")
axes[0].axhline(0.5, color="red", linestyle="--", alpha=0.5, label="H/DFA neutral = 0.5")
axes[0].axhline(0.0, color="gray", linestyle="-", alpha=0.3)
axes[0].set_xticks(x)
axes[0].set_xticklabels(labels)
axes[0].set_ylabel("Value")
axes[0].set_title("Persistence: Raw Returns")
axes[0].legend()
for bar, val in zip(bars1, vals_ret):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f"{val:.3f}", ha="center", va="bottom", fontsize=10)

# Panel 2: Absolute returns
bars2 = axes[1].bar(x, vals_abs, width, color="darkorange", alpha=0.8,
                    edgecolor="black", linewidth=0.5, label="|Returns|")
axes[1].axhline(0.5, color="red", linestyle="--", alpha=0.5, label="H/DFA neutral = 0.5")
axes[1].axhline(0.0, color="gray", linestyle="-", alpha=0.3)
axes[1].set_xticks(x)
axes[1].set_xticklabels(labels)
axes[1].set_ylabel("Value")
axes[1].set_title("Persistence: Absolute Returns (Volatility)")
axes[1].legend()
for bar, val in zip(bars2, vals_abs):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f"{val:.3f}", ha="center", va="bottom", fontsize=10)

plt.suptitle("BTC: Persistence Estimators", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 9. Cross-Asset Comparison

Load all four available assets -- BTC, SPY, Gold, and GBP/USD -- and compute
every tail and persistence estimator. This reveals which markets have the
fattest tails and strongest memory.

In [ ]:
assets = {
    "BTC":    from_sample("btc"),
    "SPY":    from_sample("spy"),
    "Gold":   from_sample("gold"),
    "GBP/USD": from_csv("data/sample/gbpusd_daily.csv"),
}

rows = []
for name, asset_df in assets.items():
    r = log_returns(asset_df)
    l = negative_returns(r)

    kap, kap_b = kappa_metric(r, n_subsamples=10)

    rows.append({
        "Asset":        name,
        "N":            len(r),
        "Hill alpha":   hill_estimator(l),
        "Kappa":        kap,
        "DEH gamma":    deh_estimator(l),
        "QQ alpha":     qq_estimator(l),
        "Pickands gamma": pickands_estimator(l),
        "MaxSum ratio": maxsum_ratio(l),
        "Hurst H":      hurst_exponent(r),
        "DFA alpha":    dfa_exponent(r),
        "Spectral d":   spectral_exponent(r),
    })

comparison = pd.DataFrame(rows).set_index("Asset")

print("Cross-Asset Comparison: All Estimators")
print("=" * 90)
display(comparison.style.format(
    {col: "{:.4f}" for col in comparison.columns if col != "N"}
).format({"N": "{:,d}"}))

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

bar_colors = ["steelblue", "seagreen", "gold", "coral"]
asset_names = comparison.index.tolist()

def make_bar(ax, col, ylabel, title, ref_line=None, ref_label=None):
    bars = ax.bar(asset_names, comparison[col], color=bar_colors,
                  alpha=0.8, edgecolor="black", linewidth=0.5)
    if ref_line is not None:
        ax.axhline(ref_line, color="red", linestyle="--", alpha=0.5,
                   label=ref_label)
        ax.legend(fontsize=9)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.tick_params(axis="x", rotation=0)
    for bar, val in zip(bars, comparison[col]):
        ax.text(bar.get_x() + bar.get_width()/2,
                bar.get_height() + 0.01 * max(abs(comparison[col])),
                f"{val:.2f}", ha="center", va="bottom", fontsize=9)

make_bar(axes[0, 0], "Hill alpha", "alpha", "Hill Tail Index",
         ref_line=2, ref_label="alpha=2")
make_bar(axes[0, 1], "Kappa", "kappa", "Kappa (Max-Stability)")
make_bar(axes[0, 2], "DEH gamma", "gamma", "DEH (Extreme Value Index)",
         ref_line=0, ref_label="gamma=0")
make_bar(axes[1, 0], "QQ alpha", "alpha", "QQ Tail Index",
         ref_line=2, ref_label="alpha=2")
make_bar(axes[1, 1], "Hurst H", "H", "Hurst Exponent",
         ref_line=0.5, ref_label="H=0.5 (random walk)")
make_bar(axes[1, 2], "DFA alpha", "alpha_DFA", "DFA Exponent",
         ref_line=0.5, ref_label="alpha=0.5 (no memory)")

plt.suptitle("Cross-Asset Comparison: Key Tail & Persistence Metrics",
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Additional metrics: Pickands, MaxSum, Spectral
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

make_bar(axes[0], "Pickands gamma", "gamma", "Pickands EVI",
         ref_line=0, ref_label="gamma=0")
make_bar(axes[1], "MaxSum ratio", "ratio", "Max-to-Sum Ratio")
make_bar(axes[2], "Spectral d", "d", "Spectral Exponent",
         ref_line=0, ref_label="d=0 (no memory)")

plt.suptitle("Cross-Asset Comparison: Additional Metrics", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 10. FRED Forex: All 23 Currency Pairs

The four sample assets above (BTC, SPY, Gold, GBP/USD) are a useful starting point,
but a sample of four cannot establish universality. Do **all** major currencies exhibit
fat tails and persistence?

We answer this using **23 daily exchange-rate series** from FRED (Federal Reserve
Economic Data), sourced via the
[forex-centuries](https://github.com/unbalancedparentheses/forex-centuries) repository.

**Data requirements:**
```bash
git clone https://github.com/unbalancedparentheses/forex-centuries ~/projects/forex-centuries
```
Or set the `FOREX_CENTURIES_DIR` environment variable to your clone location.

In [ ]:
# Load all 23 FRED forex pairs
fred_pairs = load_fred_forex()  # dict: pair_name -> DataFrame with 'close' column

print(f"Loaded {len(fred_pairs)} FRED currency pairs\n")

# Compute all 10 estimators for each pair
fred_rows = []
for pair_name, pair_df in sorted(fred_pairs.items()):
    r = log_returns(pair_df)
    l = negative_returns(r)

    if len(l) < 100:
        print(f"  Skipping {pair_name}: only {len(l)} loss observations")
        continue

    kap, kap_b = kappa_metric(r, n_subsamples=10)

    fred_rows.append({
        "Pair":          pair_name,
        "N":             len(r),
        "Hill alpha":    hill_estimator(l),
        "QQ alpha":      qq_estimator(l),
        "DEH gamma":     deh_estimator(l),
        "Pickands gamma": pickands_estimator(l),
        "Kappa":         kap,
        "MaxSum ratio":  maxsum_ratio(l),
        "Hurst H":       hurst_exponent(r),
        "DFA alpha":     dfa_exponent(r),
        "Spectral d":    spectral_exponent(r),
    })
    print(f"  {pair_name}: {len(r):,d} returns, Hill α = {fred_rows[-1]['Hill alpha']:.2f}")

fred_comp = pd.DataFrame(fred_rows).set_index("Pair")

print(f"\n{'=' * 80}")
print(f"FRED Forex: All {len(fred_comp)} Pairs — Tail & Persistence Estimators")
print(f"{'=' * 80}")
display(fred_comp.style.format(
    {col: "{:.4f}" for col in fred_comp.columns if col != "N"}
).format({"N": "{:,d}"}))

In [ ]:
# Summary statistics across all 23 pairs
summary_stats = fred_comp.drop(columns=["N"]).agg(["mean", "median", "std", "min", "max"])

print("Summary Statistics Across All FRED Forex Pairs")
print("=" * 80)
display(summary_stats.style.format("{:.4f}"))

# Key findings
n_fat = (fred_comp["Hill alpha"] < 4).sum()
n_heavy = (fred_comp["Hill alpha"] < 2).sum()
n_persist = (fred_comp["Hurst H"] > 0.5).sum()
n_deh_pos = (fred_comp["DEH gamma"] > 0).sum()

print(f"\nKey findings across {len(fred_comp)} currency pairs:")
print(f"  • Hill alpha < 4 (fat tails):       {n_fat}/{len(fred_comp)} pairs")
print(f"  • Hill alpha < 2 (infinite var):     {n_heavy}/{len(fred_comp)} pairs")
print(f"  • DEH gamma > 0 (heavy tail):        {n_deh_pos}/{len(fred_comp)} pairs")
print(f"  • Hurst H > 0.5 (persistent):        {n_persist}/{len(fred_comp)} pairs")
print(f"  • Mean Hill alpha:                    {fred_comp['Hill alpha'].mean():.2f}")
print(f"  • Mean QQ alpha:                      {fred_comp['QQ alpha'].mean():.2f}")

In [ ]:
# Heatmap: all estimators across all 23 pairs
fig, ax = plt.subplots(figsize=(16, 10))

# Normalize each column to [0, 1] for the heatmap
plot_cols = ["Hill alpha", "QQ alpha", "DEH gamma", "Pickands gamma",
             "Kappa", "MaxSum ratio", "Hurst H", "DFA alpha", "Spectral d"]
plot_data = fred_comp[plot_cols].copy()

# Z-score normalize for color mapping
plot_norm = (plot_data - plot_data.mean()) / plot_data.std()

im = ax.imshow(plot_norm.values, cmap="RdYlBu_r", aspect="auto", vmin=-2, vmax=2)
ax.set_xticks(range(len(plot_cols)))
ax.set_xticklabels(plot_cols, rotation=45, ha="right", fontsize=10)
ax.set_yticks(range(len(plot_data)))
ax.set_yticklabels(plot_data.index, fontsize=9)

# Annotate with actual values
for i in range(len(plot_data)):
    for j in range(len(plot_cols)):
        val = plot_data.iloc[i, j]
        color = "white" if abs(plot_norm.iloc[i, j]) > 1.2 else "black"
        ax.text(j, i, f"{val:.2f}", ha="center", va="center",
                fontsize=7, color=color)

plt.colorbar(im, ax=ax, label="Z-score (red = fatter tail / more persistent)")
ax.set_title("FRED Forex: All Estimators Across 23 Currency Pairs", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Distribution of Hill alpha and Hurst H across all 23 pairs
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Panel 1: Hill alpha distribution
axes[0].hist(fred_comp["Hill alpha"], bins=12, color="steelblue", alpha=0.8,
             edgecolor="black", linewidth=0.5)
axes[0].axvline(2, color="red", linestyle="--", label="alpha=2 (infinite var)")
axes[0].axvline(4, color="green", linestyle="--", label="alpha=4")
axes[0].axvline(fred_comp["Hill alpha"].mean(), color="orange", linestyle="-",
                linewidth=2, label=f"Mean = {fred_comp['Hill alpha'].mean():.2f}")
axes[0].set_xlabel("Hill alpha")
axes[0].set_ylabel("Count")
axes[0].set_title("Distribution of Hill Tail Index")
axes[0].legend(fontsize=8)

# Panel 2: Hurst H distribution
axes[1].hist(fred_comp["Hurst H"], bins=12, color="darkorange", alpha=0.8,
             edgecolor="black", linewidth=0.5)
axes[1].axvline(0.5, color="red", linestyle="--", label="H=0.5 (random walk)")
axes[1].axvline(fred_comp["Hurst H"].mean(), color="blue", linestyle="-",
                linewidth=2, label=f"Mean = {fred_comp['Hurst H'].mean():.2f}")
axes[1].set_xlabel("Hurst H")
axes[1].set_ylabel("Count")
axes[1].set_title("Distribution of Hurst Exponent")
axes[1].legend(fontsize=8)

# Panel 3: Hill alpha vs Hurst H scatter
ax3 = axes[2]
ax3.scatter(fred_comp["Hill alpha"], fred_comp["Hurst H"],
            s=60, color="steelblue", alpha=0.7, edgecolor="black", linewidth=0.5)
for pair_name in fred_comp.index:
    ax3.annotate(pair_name, (fred_comp.loc[pair_name, "Hill alpha"],
                             fred_comp.loc[pair_name, "Hurst H"]),
                 fontsize=6, textcoords="offset points", xytext=(3, 3))
ax3.axvline(2, color="red", linestyle="--", alpha=0.5)
ax3.axhline(0.5, color="red", linestyle="--", alpha=0.5)
ax3.set_xlabel("Hill alpha (lower = fatter tail)")
ax3.set_ylabel("Hurst H (higher = more persistent)")
ax3.set_title("Hill Alpha vs Hurst H")

plt.suptitle("FRED Forex: Distribution of Key Metrics Across 23 Pairs", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Summary

### All 13 estimators at a glance

| # | Estimator | Type | Key Statistic | Heavy-tail signal |
|---|-----------|------|---------------|-------------------|
| 1 | `hill_estimator` | Tail index | alpha | Low alpha (< 2 = infinite var) |
| 2 | `hill_rolling` | Rolling tail | alpha(t) | Declining alpha over time |
| 3 | `kappa_metric` | Max-stability | kappa | kappa >> Gaussian benchmark |
| 4 | `kappa_rolling` | Rolling kappa | kappa(t) | Rising kappa over time |
| 5 | `taleb_kappa` | Ratio kappa | kappa | kappa >> benchmark |
| 6 | `taleb_kappa_rolling` | Rolling Taleb | kappa(t) | Rising kappa over time |
| 7 | `pickands_estimator` | EVI | gamma | gamma > 0 (heavy tail) |
| 8 | `deh_estimator` | EVI (moments) | gamma | gamma > 0 (= 1/alpha) |
| 9 | `qq_estimator` | QQ slope | alpha | Low alpha |
| 10 | `maxsum_ratio` | Concentration | ratio | Ratio stays positive |
| 11 | `hurst_exponent` | Persistence | H | H > 0.5 |
| 12 | `dfa_exponent` | Persistence | alpha_DFA | alpha > 0.5 |
| 13 | `spectral_exponent` | Long memory | d | d > 0 |

### Key takeaways

1. **BTC has the fattest tails** among the four sample assets, consistent across
   all tail estimators (lowest Hill alpha, highest kappa, largest DEH gamma).
2. **Traditional assets** (SPY, Gold, GBP/USD) have heavier tails than
   Gaussian but are tamer than crypto.
3. **Absolute returns** show much stronger persistence than raw returns,
   confirming volatility clustering across all markets.
4. **Rolling estimates** reveal that tail heaviness is time-varying -- it
   intensifies during stress periods and relaxes in calm markets.
5. Multiple estimators provide **robustness**: if Hill, DEH, QQ, and Pickands
   all agree on heavy tails, the conclusion is credible.
6. **All 23 FRED forex pairs** confirm universality: every pair has DEH gamma > 0
   (heavy tails) and the vast majority show Hurst H > 0.5 (persistence).
   Fat tails are not an artifact of crypto or equity markets — they are a
   fundamental property of exchange rates.